<a href="https://colab.research.google.com/github/bikash-bhoi/eva4/blob/master/Session4/MNIST_Pytorch_9K_9940.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=0),  #28>26
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.2),
            nn.Conv2d(8, 16, 3, padding=0),  #26>24
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2, 2),  #24>12
            nn.Conv2d(16, 8, 1, padding=0)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(8, 8, 3, padding=0),  #12>10
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.2),
            nn.Conv2d(8, 8, 3, padding=0), #10>8
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout2d(0.1),
            nn.Conv2d(8, 16, 3, padding=0), #8>6
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            nn.Conv2d(16, 10, 6),  #6>1
            #nn.BatchNorm2d(10)
            #nn.AvgPool2d(6)
        )

        
    def forward(self, x):
        x = self.blk1(x)
        x = self.blk2(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,168
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             136
           Conv2d-11            [-1, 8, 10, 10]             584
             ReLU-12            [-1, 8, 10, 10]               0
      BatchNorm2d-13            [-1, 8, 10, 10]              16
          Dropout-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=1, gamma=0.83)


for epoch in range(1, 21):
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 1 LR: [0.1]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0496, Accuracy: 9851/10000 (99%)

Epoch: 2 LR: [0.06889]


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0196, Accuracy: 9939/10000 (99%)

Epoch: 19 LR: [0.0029005727002779317]


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9940/10000 (99%)

Epoch: 20 LR: [0.002407475341230683]


100%|██████████| 469/469 [00:10<00:00, 44.69it/s]



Test set: Average loss: 0.0190, Accuracy: 9939/10000 (99%)

